In [ ]:
# collect turtle and people counts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os

files = []
for file in os.listdir("../data/hwf_data"):
    if file.endswith(".csv"):
        files.append(os.path.join("../data/hwf_data", file))

data = pd.DataFrame()
data = data.reindex(columns = ['DateTime','TurtleNumber','PeopleNumber','Nr. Volunteers'])

for f in files:
    print(f)
    df = pd.read_csv(f, header=0)
    times = list(df.columns.values)[8:23]
    
    for i in df['DATE']:
        datetimes = pd.Series(pd.to_datetime(pd.Series([i + ' ' + t+'pm' for t in times])),name='DateTime')
        t_counts = pd.Series(np.array(df[times][df['DATE']==i].squeeze()),name='TurtleNumber')
        p_counts = pd.Series(np.array(df[[t+'.1' for t in times]][df['DATE']==i].squeeze()),name='PeopleNumber')
        v_counts = pd.Series([np.array(df.loc[df['DATE']==i,'# of Volunteers '])[0] for t in times],name='Nr. Volunteers')
        df_temp = pd.concat([datetimes, pd.to_numeric(t_counts,errors='coerce'), \
                             pd.to_numeric(p_counts,errors='coerce'), \
                             pd.to_numeric(v_counts,errors='coerce')], axis=1)
        data = data.append(df_temp)
# remove duplicate lines
data.drop_duplicates(subset='DateTime',inplace=True)
# remove lines where either the turtle or the people counts are nan
data.dropna(inplace=True)

unique_dates,cnts = np.unique(data['DateTime'],return_counts=True)

# chech if duplicate datetimes exist
if len(data['DateTime'])!= len(unique_dates):
    print('duplicate datetimes exist')
    print(len(unique_dates[cnts > 1]))
    for i in range(len(unique_dates[cnts > 1])):
        print(data[data['DateTime'] == unique_dates[cnts > 1][i]])
data.rename(columns={'Nr. Volunteers': 'Nr_Volunteers'}, inplace=True)

# sort rows by date
data.sort_values(by='DateTime',inplace=True)

print(np.shape(data))
print(np.min(data['DateTime']),np.max(data['DateTime']))

col_names = np.array(list(data.columns))

for c in col_names:
    print(c,len((data[c].unique())))
    print(data[c].unique())


In [ ]:
# function to interpolate columns
from scipy.interpolate import interp1d
from bisect import bisect

def interpolate(x_t,x_p,y_p,max_dt=2):

    x_t = np.array(x_t)
    x_p = np.array(x_p)
    y_p = np.array(y_p)
    
    sorted_indcs = np.argsort(x_p)
    x_p_sorted = x_p[sorted_indcs]
    y_p_sorted = y_p[sorted_indcs]
    
    if np.all([x_p_sorted[i+1] >= x_p_sorted[i] for i in range(len(x_p_sorted)-1)]) == False:
        print('sort failed')
        raise ValueError
    
    f = interp1d(x_p_sorted,y_p_sorted,kind='linear')
    y_t = []
    
    for t in x_t:
        indx = bisect(x_p_sorted,t)
        above = x_p_sorted[indx]
        below = x_p_sorted[indx-1]
        dt = np.abs(above - below)/3.6e12
        if dt <= max_dt:
            y_t.append(f(t))
        else:
            y_t.append(np.nan)
    
    return np.array(y_t)

In [ ]:
# tide data
files = []
for file in os.listdir("../data/sea_data"):
    if file.endswith("tide.csv"):
        files.append(os.path.join("../data/sea_data", file))
print(files)

tide = pd.DataFrame()

for f in files:
    print(f)
    df = pd.read_csv(f, header=0)
    tide = tide.append(df[['Date Time',' Water Level']])
print(np.shape(tide))
# remove duplicate rows
tide.drop_duplicates(inplace=True)
print(np.shape(tide))

x_t = pd.to_numeric(pd.to_datetime(data['DateTime']))
x_p = pd.to_numeric(pd.to_datetime(tide['Date Time']))

new_col = interpolate(x_t,x_p,tide[' Water Level'])
data['tide'] = pd.Series(new_col,index=data.index)

# add daily mean tide data
days = np.array([d[:10] for d in np.array(data['DateTime']).astype(str)])
unique_days = np.array(sorted(np.unique(days)))
daily_mean_tide = np.zeros(len(unique_days))
daily_min_tide = np.zeros(len(unique_days))
daily_max_tide = np.zeros(len(unique_days))
daily_std_tide = np.zeros(len(unique_days))

tide_days = np.array([d[:10] for d in np.array(tide['Date Time']).astype(str)])

for i in range(len(unique_days)):
    daily_mean_tide[i] = np.mean(tide.loc[tide_days == unique_days[i],' Water Level'])
    daily_max_tide[i] = np.max(tide.loc[tide_days == unique_days[i],' Water Level'])
    daily_min_tide[i] = np.min(tide.loc[tide_days == unique_days[i],' Water Level'])
    daily_std_tide[i] = np.std(tide.loc[tide_days == unique_days[i],' Water Level'])
indcs = [np.argwhere(d == unique_days)[0][0] for d in days]
data['daily mean tide'] = daily_mean_tide[indcs]
data['daily min tide'] = daily_min_tide[indcs]
data['daily max tide'] = daily_max_tide[indcs]
data['daily stdev tide'] = daily_std_tide[indcs]

plt.plot(range(100),daily_mean_tide[:100],label = 'daily mean tide')
plt.fill_between(range(100),daily_min_tide[:100],\
                 daily_max_tide[:100],alpha=0.5,label='daily min and max tide')
plt.xlabel('days')
plt.ylabel('tide height')
plt.legend()
plt.show()

plt.plot(range(100),daily_std_tide[:100])
plt.xlabel('days')
plt.ylabel('daily stdev of tide')
plt.show()

print(len(data['tide'].unique()))

col_names = np.array(list(data.columns))

for c in col_names:
    print(c,len((data[c].unique())))
    print(data[c].unique())



In [ ]:
# add daily tourism data to improve R2 of human counts

total = pd.read_excel('../data/tourism_data/daily-pax-update.xls',sheetname=0,header=3,skip_footer=13)
domestic = pd.read_excel('../data/tourism_data/daily-pax-update.xls',sheetname=1,header=2,skip_footer=13)
international = pd.read_excel('../data/tourism_data/daily-pax-update.xls',sheetname=2,header=0,skip_footer=13)

days_total = np.array([d[:10] for d in np.array(total[2018]).astype(str)])
total_counts = np.array(total['2018.1'])

days_domestic = np.array([d[:10] for d in np.array(domestic[2018]).astype(str)])
domestic_counts = np.array(domestic['Unnamed: 4'])

days_international = np.array([d[:10] for d in np.array(international['Table 1.1: International Passenger Count excluding from Canada *']).astype(str)])
international_total = np.array(international['Unnamed: 2'])
international_japan = np.array(international['Unnamed: 3'])
international_other = np.array(international['Unnamed: 4'])

date_range = np.array([d[:10] for d in pd.date_range(np.sort(days)[0],np.sort(days)[-1]).astype(str)])
print(date_range)
total_count = np.zeros([len(unique_days),7])
domestic_count = np.zeros([len(unique_days),7])
int_total_count = np.zeros([len(unique_days),7])
int_japan_count = np.zeros([len(unique_days),7])
int_other_count = np.zeros([len(unique_days),7])
for i in range(len(unique_days)):
    print(unique_days[i])
    for ii in range(7):
        indx = np.argwhere(date_range == unique_days[i])[0][0]
        print('   ',date_range[indx-(ii+1)])
        total_count[i,ii] = total_counts[days_total == date_range[indx-(ii+1)]][0]
        domestic_count[i,ii] = domestic_counts[days_domestic == date_range[indx-(ii+1)]][0]
        int_total_count[i,ii] = international_total[days_international == date_range[indx-(ii+1)]][0]
        int_japan_count[i,ii] = international_japan[days_international == date_range[indx-(ii+1)]][0]
        int_other_count[i,ii] = international_other[days_international == date_range[indx-(ii+1)]][0]
        print('   ',total_count[i,ii],domestic_count[i,ii],int_total_count[i,ii],int_japan_count[i,ii],int_other_count[i,ii])
    
indcs = [np.argwhere(d == unique_days)[0][0] for d in days]
for i in range(7):
    data['total tourists, '+str(i+1)+' day(s) ago'] = total_count[indcs,i]
    data['domestic tourists, '+str(i+1)+' day(s) ago'] = domestic_count[indcs,i]
    data['international tourists, '+str(i+1)+' day(s) ago'] = int_total_count[indcs,i]
    data['japanese tourists, '+str(i+1)+' day(s) ago'] = int_japan_count[indcs,i]
    data['other tourists,'+str(i+1)+' day(s) ago'] = int_other_count[indcs,i]

col_names = np.array(list(data.columns))

for c in col_names:
    print(c,len((data[c].unique())))
    print(data[c].unique())


In [ ]:
# add the weather data
weather = pd.read_csv('../data/weather_data/1313747_copy.csv')

col_list = list(weather.columns.values)

# collect relevant columns, interpolate and add new columns to data

cols = ['HOURLYSKYCONDITIONS','HOURLYVISIBILITY', 'HOURLYDRYBULBTEMPC', 'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempC', \
        'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection', 'HOURLYWindGustSpeed', \
        'HOURLYStationPressure', 'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting']

x_t = pd.to_numeric(pd.to_datetime(data['DateTime']))
x_p = pd.to_numeric(pd.to_datetime(weather['DATE']))
    
for c in cols:
    print(c[6:])
    
    if c == 'HOURLYSKYCONDITIONS':
        conditions = np.array([str(s)[:3] for s in weather[c]])
        conds = np.zeros(len(conditions))
        conds[conditions == 'CLR'] = 0e0
        conds[conditions == 'FEW'] = 1e0
        conds[conditions == 'SCT'] = 2e0
        conds[conditions == 'BKN'] = 3e0
        conds[conditions == 'OVC'] = 4e0
        conds[((conditions != 'CLR')&(conditions != 'FEW')&(conditions != 'SCT')&\
                   (conditions != 'BKN')&(conditions != 'OVC'))] = np.nan
        new_col = interpolate(x_t,x_p,conds)
    else:
        new_col = interpolate(x_t,x_p,weather[c])
    data[c[6:]] = pd.Series(new_col,index=data.index)
    
col_names = np.array(list(data.columns))

for c in col_names:
    print(c,len((data[c].unique())))
    print(data[c].unique())


In [ ]:
# wave data
files = []
for file in os.listdir("../data/sea_data"):
    if file.endswith("wave.txt"):
        files.append(os.path.join("../data/sea_data", file))
print(files)

wave = pd.DataFrame()

for f in files:
    print(f)
    df = pd.read_table(f, header=0,skiprows=[1],sep='\s+')
    wave = wave.append(df[['#YY','MM','DD','hh','mm','WVHT', 'DPD', 'APD', 'MWD']])
print(np.shape(wave))
# remove duplicate rows
wave.drop_duplicates(inplace=True)
print(np.shape(wave))
wave.rename(columns={'#YY': 'year', 'MM': 'month','DD': 'day','hh': 'hour','mm': 'minute'}, inplace=True)

x_t = pd.to_numeric(pd.to_datetime(data['DateTime']))
x_p = pd.to_numeric(pd.to_datetime(wave[['year','month','day','hour','minute']],errors='coerce'))

columns = ['WVHT', 'DPD', 'APD', 'MWD']
for c in columns:
    new_col = interpolate(x_t,x_p,wave[c])
    data[c] = pd.Series(new_col,index=data.index)
    
col_names = np.array(list(data.columns))

for c in col_names:
    print(c,len((data[c].unique())))
    print(data[c].unique())


In [ ]:
# write results to csv

data.rename(columns={'WVHT': 'Wave Height', 'DPD': 'Dominant Wave Period','APD': 'Average Wave Period','MWD': 'Wave Direction'}, inplace=True)
print(data)
data.to_csv('../data/combined_dataset.csv',index=False)